In [ ]:

def get_clustering(top_defect_list, method, method_kwargs, save = False, save_path = None):
    """
    
    Parameters:
    -----------
    
    Returns:
    --------
    

    """
  
    labels_list = []
    cst = method(**method_kwargs)

    for frame, defects in enumerate(top_defect_list):

        # Get defect array for frame
        defect_positions = get_defect_arr_from_frame(defects)

        if defect_positions is None:
            labels_list.append(None)
            continue

        labels = cst.fit_predict(defect_positions)
        labels_list.append(labels)

    if save:
        # save labels list
        save_path = save_path if save_path is not None else 'labels_list.pkl'
        with open(save_path, 'wb') as f:
            pkl.dump(labels_list, f)

    return labels_list



In [ ]:
dist_max = [10, 20, 30,]
output_path= 'test'
if not os.path.exists(output_path):
    os.makedirs(output_path)

for dist in dist_max:
    save_path = os.path.join(output_path, f'labels_rm{dist}.pkl')
    method_kwargs = dict(n_clusters=None, linkage = 'single', distance_threshold=dist)
    _ = get_clustering(defect_list, AgglomerativeClustering, method_kwargs, save = True, save_path=save_path)


In [ ]:
f_idx = 4
defect_arr = get_defect_arr_from_frame(defect_list[f_idx])

# save defect_list as pkl
import pickle as pkl
with open('defect_list.pkl', 'wb') as f:
    pkl.dump(defect_list, f)


# load defect_list.npz
defect_dict = np.load('defect_list.npz', allow_pickle=True)


defect_positions = np.empty([len(defect_list[0]), 3], dtype='float')

for i, defs in enumerate(defect_list[0]):
    print(defs['pos'], defs['charge'])
    defect_positions[i] = *defs['pos'], defs['charge']


arr_dict = {}
for fidx, defects in enumerate(defect_list):
    Ndefects = len(defects)
    if Ndefects == 0:
        arr_dict[f'fidx_{fidx}'] = np.array([0,0,0], dtype ='int')
    defect_positions = np.zeros([Ndefects, 3], dtype='float32')
    for i, defect in enumerate(defects):
        defect_positions[i] = *defect['pos'], defect['charge'] * 2
    defect_positions[:, :2] -= .5
    arr_dict[f'fidx_{fidx}'] = defect_positions.astype('int')

In [ ]:

def calc_distance(points, L, periodic = False):
    # points: (N,3)-array of (x,y,z) coordinates for N points
    # distance(points): returns (N,N)-array of inter-point distances:

    ## points[NA,:] creates a 1,N,3 array of points, i.e. copies of the matrix points along first axis points[NA,:] = points[NA,N,3]=[points,ponts,points,...]
    ## points[:,NA] creates an N,1,3 array. points[:,NA]=points[N,NA,3] = [x1, x2, ..., xn]
    ## subtracting the two makes a N,N,3 matrix, where each copy of points is subtracted by each row of x1
    ## that is, for points[:,NA] = points[:,NA,:], we will copy along the rows. So row one, x1, will subtracted from the
    ## 1st NA entry in points[NA,:], which is just points, yielding points-x1*Identity.

    ## ie displacement = (points-x1*Id,points-x2*Id,...,points-x_n*Id) = ([x1-x1;x2-x1,...,xN-x1],[x1-x2;x2-x2,...xN-xn],...,[x1-xN;...;xN-xN]
    ## summing displacement*2 over axis -1 (i.e last row ie the columns) yields then an NxN matrix
    ## sum (discplacement**2,axis=2) = [(x1-x1)^2, (x1-x2)^2, ... (x1-xN)^2; (x2-x1)^2,(x2-x1)^2,...
    ## this producing an NxN matrix of distances between all pairs of particles. Only Upper half contains non-redundant info

    displacement = np.zeros((points.shape[0], points.shape[0], 2))
    displacement_direct = np.abs(points[:, None] - points[None, :])

    if periodic:
        displacement_periodic = L - displacement_direct
        displacement[:,:,0] = np.concatenate([displacement_direct[:,:,0, None], displacement_periodic[:,:,0, None]], axis=-1).min(axis=-1)
        displacement[:,:,1] = np.concatenate([displacement_direct[:,:,1, None], displacement_periodic[:,:,1, None]], axis=-1).min(axis=-1)
    else:
        displacement = displacement_direct

    return np.sqrt(np.sum(displacement*displacement, axis=-1))

In [ ]:
nrows = 2
fig, ax = plt.subplots(nrows = nrows, ncols = 2, figsize=(14,7*nrows))
ax = ax.flatten()
f = get_frame_number(f_idx, paths[N], ar.ninfo)
frame=ar._read_frame(f)
mp.nematic.plot.director(frame, ax[0]);
mp.nematic.plot.defects(frame, ax[0]);

dist_max = [25, 30, 35]
for i, dist in enumerate(dist_max):
    method_kwargs = dict(n_clusters=None, linkage = 'single', distance_threshold=dist)
    single_cluster  = AgglomerativeClustering(metric = lambda arr: calc_distance(arr, ar.LX, periodic = False), n_clusters=None, linkage = 'single', distance_threshold=dist).fit_predict(defect_arr)
  #  ax[i+1].fill(x, y, facecolor='lightsalmon', edgecolor='orangered', linewidth=3)
   # ax[i + 1].scatter(defect_arr[:,0], defect_arr[:,1], c=single_cluster, cmap= 'tab20', s=8, lw=2)
   # ax[i + 1].plot(defect_arr[:,0], defect_arr[:,1], 'o', c=single_cluster, markersize=8, cmap= 'tab20',  alpha=0.5)
    vals = np.unique(single_cluster)
    ax[i+1].set_title(f'Rmax = {dist}')

    color_list = ['blue', 'orange', 'purple', 'pink', 'yellow', 'brown', 'grey', 'cyan', 'magenta']
    marker_list = ['+', 's', 'd', 'v', '^', '<', '>', 'p',]
    for val in vals:
        idx = np.where(single_cluster == val)
      #  print(len(idx[0]))
        Nmembers = len(idx[0])
        if Nmembers == 1:
            color = 'black'
            marker = '.'
        elif Nmembers == 2:
            color = 'red'
            marker = 'o'
        elif Nmembers > 10:
            color = 'green'
            marker = 'x'
        else:
            color = color_list[Nmembers - 3]
            marker = marker_list[Nmembers - 3]

        ax[i+1].scatter(defect_arr[idx,0], defect_arr[idx,1], c = color, marker = marker, s=35)
fig.text(0.05, .975, f'n=1 ', fontsize=14, verticalalignment='bottom', color='black', fontweight='bold')
fig.text(0.1, .975, f'n=2 ', fontsize=14, verticalalignment='bottom', color='red', fontweight='bold')
for i in range(3, 10):
    fig.text(i*0.05, .975, f'n={i} ', fontsize=14, verticalalignment='bottom', color=color_list[i-3], fontweight='bold')
#fig.text(0.3, 1.015, f'n=3 ', fontsize=14, verticalalignment='bottom', color=color_list[3-3], fontweight='bold')
fig.text(10*0.05, .975, f'n>10 ', fontsize=14, verticalalignment='bottom', color='green', fontweight='bold')

In [ ]:
fig, ax = plt.subplots(nrows = 3, ncols = 2, figsize=(10,15))
ax = ax.flatten()
frame=ar._read_frame(f)
mp.nematic.plot.director(frame, ax[0]);
mp.nematic.plot.defects(frame, ax[0]);
ax[1].scatter(defect_arr[:,0], defect_arr[:,1], c=ap_clustering, cmap='viridis', s=8)
ax[2].scatter(defect_arr[:,0], defect_arr[:,1], c=ward_cluster, cmap='viridis', s=8)
ax[3].scatter(defect_arr[:,0], defect_arr[:,1], c=av_cluster, cmap='viridis', s=8)
ax[4].scatter(defect_arr[:,0], defect_arr[:,1], c=single_cluster, cmap= 'tab20c_r', s=8)
mp.nematic.plot.director(frame, ax[-1]);
mp.nematic.plot.defects(frame, ax[-1]);

In [ ]:
ax[1].scatter(defect_arr[:,0], defect_arr[:,1], c=ap_clustering, cmap='viridis', s=8)
ax[2].scatter(defect_arr[:,0], defect_arr[:,1], c=ward_cluster, cmap='viridis', s=8)
fig